<a href="https://colab.research.google.com/github/lamld203844/chat-any/blob/main/chat_any.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# System flow

In [1]:
import nest_asyncio

# allows nested access to the event loop
nest_asyncio.apply()

In [2]:
!pip install -q langchain
!pip install -q langchain-community
!pip install -q llama-index

!pip install -q llama-index-embeddings-langchain
!pip install -q sentence-transformers

!pip install -q llama-index-llms-gemini
!pip install -q google-generativeai
!pip install -q llama-index-readers-web

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load file
- load website

In [3]:

# -------------------------------------------
# Load data from a website via Llamaindex Loader
#
# -------------------------------------------

from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

docs = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://cinnamon.is/en/"]
)


## Chunking and creating embeddings model

In [7]:

# --------------------------------------------
# Chunking and create embeddings
# Automatic via llama index VectorStoreIndex
# --------------------------------------------
from torch import cuda
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

def load_embedding_model(
    model_name: str = "BAAI/bge-large-en-v1.5",
    device: str = "cuda" if cuda.is_available() else "cpu"
) -> HuggingFaceBgeEmbeddings:
    model_kwargs = {"device": device}
    encode_kwargs = {
        "normalize_embeddings": True
    }  # set True to compute cosine similarity
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embedding_model

# setting up the embedding model
lc_embedding_model = load_embedding_model()
embed_model = LangchainEmbedding(lc_embedding_model)

### Sanity check embedding model
embedding = lc_embedding_model.embed_query('Hello, world')
embedding = embedding[:10]
print(f'Embedding: {embedding}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embedding: [0.0570748969912529, 0.002089085290208459, 0.031196100637316704, -0.02522590011358261, 0.011089175008237362, -0.0145485894754529, 0.03243298456072807, 0.02435419335961342, 0.012902908958494663, 0.047478292137384415]


In [8]:
# ====== Create vector store and upload data ======
Settings.embed_model = lc_embedding_model
index = VectorStoreIndex.from_documents(docs, show_progress=True)
# TODO try async index creation for faster emebdding generation & persist it to memory!
# index = VectorStoreIndex(docs, use_async=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

## Load llm

In [15]:
# --------------------------
# Set up search query engine
# --------------------------

# setting up the llm
from llama_index.llms.gemini import Gemini
from google.colab import userdata

google_api = userdata.get('GOOGLE_API_KEY')
llm = Gemini(model_name="models/gemini-pro", api_key=google_api)

# Sanity check llm
resp = llm.complete("Hello, world")
print(resp)

Hello, world! I am a large language model, trained by Google.


# Set up query engine

In [10]:

# ====== Setup a query engine ======
Settings.llm = llm
query_engine = index.as_query_engine(similarity_top_k=4)

# ---------------------------------------
# Customise prompt template + augmenting
# ---------------------------------------

from llama_index.core import PromptTemplate

qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [11]:
# ---------- Chatting -----------
from IPython.display import Markdown, display
response = query_engine.query('What is this website about?')
display(Markdown(str(response)))

This website is about Cinnamon, an AI company that provides deep learning backed AI products.

# GUI with Streamlit


In [12]:
!pip install -q streamlit
!npm install -q localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.219s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [16]:
%%writefile app.py

import os
import gc

import uuid
import streamlit as st
from torch import cuda
# from dotenv import load_dotenv
# load_dotenv() # Load Gemini API
from google.colab import userdata


from llama_index.core import Settings
from llama_index.core import PromptTemplate
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from llama_index.llms.gemini import Gemini


# ---------- Init + Helper function ----------------

# setting up the embedding model
def load_embedding_model(
    model_name: str = "BAAI/bge-large-en-v1.5",
    device: str = "cuda" if cuda.is_available() else "cpu"
) -> HuggingFaceBgeEmbeddings:
    model_kwargs = {"device": device}
    encode_kwargs = {
        "normalize_embeddings": True
    }  # set True to compute cosine similarity
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embedding_model

lc_embedding_model = load_embedding_model()
embed_model = LangchainEmbedding(lc_embedding_model)

# setting up session
if "id" not in st.session_state:
    st.session_state.id = uuid.uuid4()
    st.session_state.file_cache = {}

session_id = st.session_state.id
client = None

# setting up the llm
google_api = userdata.get('GOOGLE_API_KEY')
llm = Gemini(model_name="models/gemini-pro", api_key=google_api)

# Sanity check llm
resp = llm.complete("Hello, world")
print(resp)
# helper func
def reset_chat():
    st.session_state.messages = []
    st.session_state.context = None
    gc.collect()

def validate_website_url(url):
    return True

# ---------- End helper function ----------------

with st.sidebar:
    # Input for URL
    website_url = st.text_input("URL")

    # Button to load and process url
    process_button = st.button("Load")

    message_container = st.empty()  # Placeholder for dynamic messages

    if validate_website_url(website_url):
        with st.spinner(f"Loading website..."):
            try:
                # -------------------------------------------
                # Load data from a website via Llamaindex Loader
                # -------------------------------------------
                docs = SimpleWebPageReader(html_to_text=True).load_data([website_url])

                # ---- Create vector store and upload data ---
                # Chunking and create embeddings
                # Automatic via llama index VectorStoreIndex
                # --------------------------------------------
                Settings.embed_model = embed_model
                index = VectorStoreIndex.from_documents(docs)

                # ====== Setup a query engine ======
                Settings.llm = llm
                query_engine = index.as_query_engine(streaming=True, similarity_top_k=4)

                # ====== Customise prompt template ======
                qa_prompt_tmpl_str = (
                "Context information is below.\n"
                "---------------------\n"
                "{context_str}\n"
                "---------------------\n"
                "Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
                "Query: {query_str}\n"
                "Answer: "
                )
                qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

                query_engine.update_prompts(
                    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
                )

                if docs:
                    message_container.success("Data loaded successfully!!")
                else:
                    message_container.write(
                        "No data found, check if the repository is not empty!"
                    )
                st.session_state.query_engine = query_engine

            except Exception as e:
                st.error(f"An error occurred: {e}")
                st.stop()

            st.success("Ready to Chat!")
    else:
        st.error('Invalid url')
        st.stop()

col1, col2 = st.columns([6, 1])

with col1:
    st.header(f"Chat with website")

with col2:
    st.button("Clear ↺", on_click=reset_chat)


# Initialize chat history
if "messages" not in st.session_state:
    reset_chat()


# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])


# Accept user input
if prompt := st.chat_input("What's up?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""

        # context = st.session_state.context
        query_engine = st.session_state.query_engine

        # Simulate stream of response with milliseconds delay
        streaming_response = query_engine.query(prompt)

        for chunk in streaming_response.response_gen:
            full_response += chunk
            message_placeholder.markdown(full_response + "▌")

        # full_response = query_engine.query(prompt)

        message_placeholder.markdown(full_response)
        # st.session_state.context = ctx

    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": full_response})

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.132.238.61
npx: installed 22 in 3.463s
your url is: https://large-geckos-jog.loca.lt
